## Project outline 
- we're going to scrape first five pages of  https://news.ycombinator.com/news
- we'll get the list of articles. For each article, we'll get link of article, title, number of comments, published date of article, author of that article
- Firstly, we'll scrape only first page of site "hacker news" after we will employe web crawler technique to scrape more pages 
- After scraping, we'll convert it in csv file
- In the begining we need to import requests and BeautifulSoup library.
- BeautifulSoup takes the HTMl file and parse it into tree like structure

In [1]:
import requests 
import re 
from bs4 import BeautifulSoup

- we have created different lists to store different elements like link, title, author, score, comments
- Then we'll create a dictionary to store the list and create a dataframe out of dictionary
- In last we have stored the dataframe as csv file in base directory

- The code is for scraping first page of hackernews site

url = 'https://news.ycombinator.com/news'
response =requests.get(url)
a_link, a_name, a_author, a_score,a_date, a_comment =[],[],[],[],[],[]
doc =BeautifulSoup(response.text, 'html.parser')
for item in doc.find_all('tr', class_= 'athing'):
    a_tag =item.find('a', class_='storylink')
    link =a_tag.get('href') if a_tag else None 
    a_link.append(link)
    article_name =a_tag.get_text(strip=True) if a_tag else None
    a_name.append(article_name)
for var in doc.find_all('td', class_='subtext'):
    author =var.find('a', class_='hnuser')
    author =author.get_text(strip=True) if author else 'missing'
    a_author.append(author)
    score =var.find('span', class_ ='score')
    score =score.get_text(strip =True) if score else '0 points'
    a_score.append(score)
    date =var.find('span' ,class_ ='age')
    date =date.get('title') if date else None
    a_date.append(date)
    comments =var.find('a', string=re.compile(r'\d+(&nbsp;|\s)comment(s?)'))
    comment =comments.get_text(strip=True).replace('\xa0','')  if comments else '0 comment'
    a_comment.append(comment)

- The code is crawling multiple pages, however we have only crawled first 3 pages, one can get more scraped pages just changing the range(a,b) in the 12 line of code.

In [4]:
url = 'https://news.ycombinator.com/news'
def site_crawler1(url):  # site_crawler1 take care of link of article and title of article
    response =requests.get(url)
    doc =BeautifulSoup(response.text, 'html.parser')
    return doc.find_all('a' ,class_='storylink')    
def site_crawler2(url): # this function takes care of author, date, comment, score of the article
    res= requests.get(url)
    doc =BeautifulSoup(res.text, 'html.parser')
    return doc.find_all('td', class_='subtext')
a_link, a_name, a_author, a_score,a_date, a_comment =[],[],[],[],[],[] # here we have defined multiple list to store the
                                                                       # data like a_link store all the links
for i in range(1,4):
    for item in site_crawler1(url+'?p='+str(i)):  # here url is given of site +'?p='+str(i) used for jumping to pages 
        a_link.append(item.get('href') if item else None)   # here we have collected all link using href tag
        a_name.append(item.get_text(strip=True) if item else None)
    for var in site_crawler2(url+'?p='+str(i)):  
        author =var.find('a', class_='hnuser')  # here we are trying to find a tag with hunser class which includes all
                                           # details of article like score, publishing date, no of comment, article'author
## here we have defined different function for different work one can define only one function for parent tag and can use 
## find_sibling() or find_next_sibling() function to reduce the code length. however, i tried to do so but getting error 
        a_author.append(author.get_text(strip=True) if author else 'missing')
        score =var.find('span', class_ ='score')
        a_score.append(score.get_text(strip =True) if score else '0 points')
        date =var.find('span' ,class_ ='age')
        date =date.get('title') if date else None
        a_date.append(date)
        comments =var.find('a', string=re.compile(r'\d+(&nbsp;|\s)comment(s?)')) # here we have used regex expression since
        ## comments section has tag which are similar to other tag so we need to take care of that to get particular 
        ## section (i.e.) comments in the a tag with class hunser
        comment =comments.get_text(strip=True).replace('\xa0','')  if comments else '0 comment'
        a_comment.append(comment)
article ={'title':a_name,'author':a_author,'link':a_link,'score':a_score,'date':a_date,'comment':a_comment}
 ## here we created a dictionary although one can use zip method to merge the list but need to explicitly defince the 
    ## columns name while create dataframe
import pandas as pd
df =pd.DataFrame(article)
df.to_csv('hackernews123.csv', sep=',', index=False)

In [5]:
df_new =pd.read_csv('hackernews123.csv')

In [8]:
pd.set_option('Display.max_rows', None)  # display all the rows in the dataframe
df_new

,title,author,link,score,date,comment
0,US companies hit by 'colossal' cyber-attack,sedeki,https://www.bbc.com/news/world-us-canada-57703836,174 points,2021-07-03T01:08:21,115comments
1,The Untold Story of SQLite with Richard Hipp,signa11,https://corecursive.com/066-sqlite-with-richar...,68 points,2021-07-03T01:49:37,11comments
2,Brickit – scans your Lego bricks and helps you...,ChrisArchitect,https://twitter.com/AlexanderNL/status/1410253...,414 points,2021-06-30T21:23:43,67comments
3,Soldiers watch the US withdrawal from Bagram t...,lalaland1125,https://www.stripes.com/theaters/middle_east/2...,88 points,2021-07-02T23:28:54,34comments
4,An Oral History of the L0pht (2018),whiteyford,https://duo.com/decipher/an-oral-history-of-th...,93 points,2021-07-02T23:02:29,6comments
5,Another 0-day looms for many Western Digital u...,danso,https://krebsonsecurity.com/2021/07/another-0-...,327 points,2021-07-02T16:14:16,186comments
6,Rome's Colosseum opens its underground for the...,riffraff,https://www.cnn.com/travel/article/colosseum-r...,54 points,2021-07-02T04:42:46,1comment
7,Towards Richer Colors on the Web,asicsp,https://darker.ink/writings/Towards-richer-col...,28 points,2021-07-02T10:19:41,2comments
8,“Space age electronics”: Inside a GE thin-film...,parsecs,https://www.righto.com/2021/07/space-age-elect...,110 points,2021-07-02T19:37:46,30comments
9,Back to the Future with RSS,Zhyl,https://ncase.me/rss/,123 points,2021-07-02T09:36:05,34comments
